# Head

In [1]:
from google.colab import drive
import os
drive.mount('/content/gdrive')
os.chdir("/content/gdrive/Othercomputers/retep's PC/MambaVAE")
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

Mounted at /content/gdrive


In [ ]:
%pip install datasets causal_conv1d

# Lab

In [ ]:
import VAE, importlib
importlib.reload(VAE)
from VAE import MambaVAE

In [ ]:
import torch, InitMamba
from transformers import MambaConfig, MambaForCausalLM, AutoTokenizer

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("state-spaces/mamba-130m-hf")
inputs = tokenizer(["Today is Monday, Tomorrow is Tuesday."], return_tensors='pt').to('cuda')

In [ ]:
model1 = InitMamba.MambaForCausalLM.from_pretrained("state-spaces/mamba-130m-hf").cuda()
model2 = InitMamba.MambaModel.from_pretrained("state-spaces/mamba-130m-hf").cuda()
# model1.train()
# model2.train()

hidden_states = model2(**inputs, layer_range = range(23))[0]
res1 = model1(inputs_embeds = hidden_states, attention_mask = inputs['attention_mask'], layer_range = range(23, 24), output_ssm_last_states = True)
# res1 = model1(**inputs, output_ssm_last_states = True)
ssm_last_states = res1.ssm_last_states
print(ssm_last_states)

In [ ]:
hidden_states = model2(**inputs, layer_range = range(1), inputs_ssm_states = ssm_last_states)[0]
res1 = model1(inputs_embeds = hidden_states, attention_mask = inputs['attention_mask'],
              layer_range = range(1, 24),
              output_ssm_last_states = True,
              inputs_ssm_states = ssm_last_states)
res = model1(**inputs,
             output_ssm_last_states = True,
             inputs_ssm_states = ssm_last_states)
print(res.logits-res1.logits)
print(res.logits)

In [13]:
!python Trainer.py

The fast path is not available because one of `(selective_state_update, selective_scan_fn, causal_conv1d_fn, causal_conv1d_update, mamba_inner_fn)` is None. Falling back to the sequential implementation of Mamba, as use_mambapy is set to False. To install follow https://github.com/state-spaces/mamba/#installation and https://github.com/Dao-AILab/causal-conv1d. For the mamba.py backend, follow https://github.com/alxndrTL/mamba.py.
/content/gdrive/Othercomputers/retep's PC/MambaVAE/Trainer.py:52: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. A